[View in Colaboratory](https://colab.research.google.com/github/ajdunlap/2DKPZcalculations/blob/master/variancederivs_simpler3.ipynb)

In [0]:
from sympy import *
from sympy.utilities.iterables import *
from collections import Counter, defaultdict
import itertools
import math
import copy
eps = symbols('varepsilon')
def Sn(n):
    return [str(j) + "A" for j in range(n)] + [str(j) + "B" for j in range(n)]

def make_pis(n,k):
    return multiset_partitions(Sn(n),m=k)

def make_TB(B,j):
    x = itertools.product(*([[0]]+[[0,1]]*j+[[0]]*(len(B)-j-1)))
    #x = itertools.product(*([[0,1]]*len(B)))
    for k,y in enumerate(x):
        rv = list(y)
        rv[0] = (rv[0] + sum(y)) % 2
        yield ''.join(map(str,rv))
        
def make_TB_full(B):
    #x = itertools.product(*([[0]]+[[0,1]]*j+[[0]]*(len(B)-j-1)))
    x = itertools.product(*([[0,1]]*len(B)))
    for k,y in enumerate(x):
        rv = list(y)
        if sum(rv) % 2 == 0:
          yield ''.join(map(str,rv))

def makeInThePatternOf(nestedl,flatl):
    j = 0
    rv = []
    for x in nestedl:
        toadd = []
        for y in x:
            toadd.append(flatl[j])
            j += 1
        rv.append(toadd)
    return rv

def make_as(pi):
    xs = itertools.product(*itertools.chain(*[[make_TB_full(B) for j in range(len(B))] for B in pi]))
    return (makeInThePatternOf(pi,x) for x in xs)

def locate_everyone(pi):
    rv = [[None,None] for z in range(sum(len(x) for x in pi)//2)]
    for j,x in enumerate(pi):
        for k,y in enumerate(x):
            rv[ord(y[0])-ord('0')][ord(y[1])-ord('A')] = (j,k)
    rv = [(x,y) for [x,y] in rv]
    return rv

def make_product(pi,a):
    rv = []
    xs = locate_everyone(pi)
    for j in xs:
        y = []
        for x in j:
            t = (x[0],a[x[0]][x[1]],x[1],(-1)**(ord(a[x[0]][x[1]][x[1]])-ord('0')))
            y.append(t)
        rv.append(tuple(y))
    return rv

def is_split(B):
    for x in B:
        y = list(x)
        y[1] = 'B' if y[1] == 'A' else 'A'
        if ''.join(y) not in B:
            return True
    return False

def compute_sign(a):
    z = 0
    for x in a:
        for j,y in enumerate(x):
            z += ord(y[j]) - ord('0')
    return (-1)**z

def all_split(p):
    return all([is_split(B) for B in p])

def make_contributions(pis):
    contributions = []
    n = 0
    for p in pis:
        my_as = make_as(p)
        for a in my_as:
            n += 1
            produ = make_product(p,a)
            contribution = Counter([((x[0][0],x[0][1]),(x[1][0],x[1][1])) for x in produ])
            contributions.append((produ,contribution,ev(contribution)
                                  ,compute_sign(a),a))
    return contributions

def evaluate_one(p,v):
    x,y = p
    if x[0] == y[0]:
        if x[1] == y[1]:
            return eps**(-2*v)
        else:
            return math.factorial(v)*(log(eps))**v
    else:
        return math.factorial(v-1)*(log(eps))**(v-1)
    
def ev(c):
    rv = 1
    for k,v in c.items():
        rv *= evaluate_one(k,v)
    return rv
  
def make_graphs(p):
  return itertools.product(*[list(multiset_partitions(B)) for B in p])

def find_everyone_in_graph(p,g):
  rv = [[None,None] for z in range(sum(len(x) for x in p)//2)]
  for j,B in enumerate(g):
    n = 0
    for k,BB in enumerate(B):
      for l,y in enumerate(BB):
        rv[ord(y[0])-ord('0')][ord(y[1])-ord('A')] = (j,k,n)
        n += 1
  return rv

def make_as_for_graph(p,g):
  #rv = copy.deepcopy(g)
  rv = []
  for k,B in enumerate(g):
    rv.append(itertools.permutations(make_TB_full(p[k]),len(B)))
    #for j,BB in enumerate(B):
    #  rv[k][j] = list(make_TB_full(p[k]))
  #return itertools.product(*[itertools.product(*B) for B in rv])
  return itertools.product(*rv)
  
def assign_as(p,g,aslist):
  places = find_everyone_in_graph(p,g)
  return makeInThePatternOf(places,[(k,aslist[k][j],l) for pls in places for (k,j,l) in pls ])

def graph_as_sign(assigned_as):
  return (-1)**sum(sum(ord(x[1][x[2]])-ord('0') for x in l) for l in assigned_as)

def graph_contrib(p,g):
  my_as = make_as_for_graph(p,g)
  rv = 0
  first = True
  score = 0
  for a in my_as:
    assigned_as = assign_as(p,g,a)
    sign = graph_as_sign(assigned_as)
    if first:
      to_score = [tuple([tuple(x[0:2]) for x in y]) for y in assigned_as]
      score = ev(Counter(to_score))
      first = False
    rv += sign
  return (rv,score)

def make_actual_graph
  
#def make_contributions_for_graph(pi,g):
#    contributions = []
#    for p in pis:
#        my_as = make_as_for_graph(p,g)
#        for a in my_as:
#          prods = 
#            produ = make_product(p,a)
#            contribution = Counter([((x[0][0],x[0][1]),(x[1][0],x[1][1])) for x in produ])
#            contributions.append((produ,contribution,ev(contribution),compute_sign(a),a))
#    return contributions

In [0]:
ps = list(p for p in make_pis(2,2) if all_split(p))


In [77]:
rv = 0
n = 3
k = 2
ps = list(p for p in make_pis(n,k) if all_split(p))
for p in ps:
  print (p)
  gs = make_graphs(p)
  for g in gs:
    x = graph_contrib(p,g)
    #print ('  ',g,2**(-2*n)*math.factorial(k)*x[0],x[1])
    rv += abs(2**(-2*n+1)*math.factorial(k)*x[0]*x[1])
print(rv)

[['0A', '0B', '1A', '1B', '2A'], ['2B']]
[['0A', '0B', '1A', '1B', '2B'], ['2A']]
[['0A', '0B', '1A', '2A', '2B'], ['1B']]
[['0A', '0B', '1A', '2A'], ['1B', '2B']]
[['0A', '0B', '1A', '2B'], ['1B', '2A']]
[['0A', '0B', '1A'], ['1B', '2A', '2B']]
[['0A', '0B', '1B', '2A', '2B'], ['1A']]
[['0A', '0B', '1B', '2A'], ['1A', '2B']]
[['0A', '0B', '1B', '2B'], ['1A', '2A']]
[['0A', '0B', '1B'], ['1A', '2A', '2B']]
[['0A', '0B', '2A'], ['1A', '1B', '2B']]
[['0A', '0B', '2B'], ['1A', '1B', '2A']]
[['0A', '1A', '1B', '2A', '2B'], ['0B']]
[['0A', '1A', '1B', '2A'], ['0B', '2B']]
[['0A', '1A', '1B', '2B'], ['0B', '2A']]
[['0A', '1A', '1B'], ['0B', '2A', '2B']]
[['0A', '1A', '2A', '2B'], ['0B', '1B']]
[['0A', '1A', '2A'], ['0B', '1B', '2B']]
[['0A', '1A', '2B'], ['0B', '1B', '2A']]
[['0A', '1A'], ['0B', '1B', '2A', '2B']]
[['0A', '1B', '2A', '2B'], ['0B', '1A']]
[['0A', '1B', '2A'], ['0B', '1A', '2B']]
[['0A', '1B', '2B'], ['0B', '1A', '2A']]
[['0A', '1B'], ['0B', '1A', '2A', '2B']]
[['0A', '2A', '2

In [17]:
#find_everyone_in_graph(p,g)
assign_as(p,g,list(make_as_for_graph(p,g))[2])

[[(0, '000', 0), (0, '000', 1)], [(0, '101', 2), (1, '0', 0)]]

In [18]:
graph_sign(assign_as(p,g,list(make_as_for_graph(p,g))[2]))

-1

In [11]:
for k,p in enumerate(ps):
  print("step",k)
  cs = make_contributions([p])
  print(sum(x[2]*x[3] for x in cs))

step 0
0
step 1
0
step 2
0
step 3
0
step 4
0
step 5
0
step 6
0
step 7
0
step 8
0
step 9
0
step 10
0
step 11
0
step 12
0
step 13
0
step 14
0
step 15
0
step 16
0
step 17
2*log(varepsilon)**2 - 3*log(varepsilon) + 2
step 18
0
step 19
0
step 20
0
step 21
0
step 22
0
step 23
0
step 24
0
step 25
0
step 26
0
step 27
0


In [0]:
myp = [['0A','1A','2A','2B','3A','3B'],['0B','1B']]

In [0]:
def twoandtwo(l):
  ll = l
  return (ll[0] == ll[2] and ll[1] == ll[3] and ll[0] != ll[1])
l = [x for x in cs if x[0][0][0][1] == x[0][1][0][1] and x[0][0][1][1] == x[0][1][1][1] and twoandtwo([x[0][2][0][1],x[0][2][1][1],x[0][3][0][1],x[0][3][1][1]])]

In [64]:
l

[([((0, '000000', 0, 1), (1, '00', 0, 1)),
   ((0, '000000', 1, 1), (1, '00', 1, 1)),
   ((0, '000000', 2, 1), (0, '100100', 3, -1)),
   ((0, '000000', 4, 1), (0, '100100', 5, 1))],
  Counter({((0, '000000'), (0, '100100')): 2, ((0, '000000'), (1, '00')): 2}),
  2*log(varepsilon)**3,
  -1),
 ([((0, '000000', 0, 1), (1, '00', 0, 1)),
   ((0, '000000', 1, 1), (1, '00', 1, 1)),
   ((0, '000000', 2, 1), (0, '101000', 3, 1)),
   ((0, '000000', 4, 1), (0, '101000', 5, 1))],
  Counter({((0, '000000'), (0, '101000')): 2, ((0, '000000'), (1, '00')): 2}),
  2*log(varepsilon)**3,
  1),
 ([((0, '000000', 0, 1), (1, '00', 0, 1)),
   ((0, '000000', 1, 1), (1, '00', 1, 1)),
   ((0, '000000', 2, 1), (0, '001100', 3, -1)),
   ((0, '000000', 4, 1), (0, '001100', 5, 1))],
  Counter({((0, '000000'), (0, '001100')): 2, ((0, '000000'), (1, '00')): 2}),
  2*log(varepsilon)**3,
  -1),
 ([((0, '000000', 0, 1), (1, '00', 0, 1)),
   ((0, '000000', 1, 1), (1, '00', 1, 1)),
   ((0, '000000', 2, 1), (0, '110000', 3

In [0]:
myp2 = [['0A','0B','1A','1B']]

In [89]:
cs2 = make_contributions([myp2])
l2 = [x for x in cs2 if  twoandtwo([x[0][0][0][1],x[0][0][1][1],x[0][1][0][1],x[0][1][1][1]])]
sum(x[2]*x[3] for x in l2)

-16*log(varepsilon)**2

In [102]:
for x in l2:
  print(x[0][0][0][1],'/',x[0][0][0][3],x[0][0][1][1],'/',x[0][0][1][3],x[0][1][0][1],'/',x[0][1][0][3],x[0][1][1][1],'/',x[0][1][1][3])

0000 / 1 0011 / 1 0000 / 1 0011 / -1
0000 / 1 0101 / -1 0000 / 1 0101 / -1
0000 / 1 0110 / -1 0000 / 1 0110 / 1
0000 / 1 1001 / 1 0000 / 1 1001 / -1
0000 / 1 1010 / 1 0000 / 1 1010 / 1
0000 / 1 1100 / -1 0000 / 1 1100 / 1
0000 / 1 1111 / -1 0000 / 1 1111 / -1
0011 / 1 0000 / 1 0011 / -1 0000 / 1
0011 / 1 0101 / -1 0011 / -1 0101 / -1
0011 / 1 0110 / -1 0011 / -1 0110 / 1
0011 / 1 1001 / 1 0011 / -1 1001 / -1
0011 / 1 1010 / 1 0011 / -1 1010 / 1
0011 / 1 1100 / -1 0011 / -1 1100 / 1
0011 / 1 1111 / -1 0011 / -1 1111 / -1
0101 / 1 0000 / 1 0101 / 1 0000 / 1
0101 / 1 0011 / 1 0101 / 1 0011 / -1
0101 / 1 0110 / -1 0101 / 1 0110 / 1
0101 / 1 1001 / 1 0101 / 1 1001 / -1
0101 / 1 1010 / 1 0101 / 1 1010 / 1
0101 / 1 1100 / -1 0101 / 1 1100 / 1
0101 / 1 1111 / -1 0101 / 1 1111 / -1
0110 / 1 0000 / 1 0110 / -1 0000 / 1
0110 / 1 0011 / 1 0110 / -1 0011 / -1
0110 / 1 0101 / -1 0110 / -1 0101 / -1
0110 / 1 1001 / 1 0110 / -1 1001 / -1
0110 / 1 1010 / 1 0110 / -1 1010 / 1
0110 / 1 1100 / -1 0110 / -

In [103]:
len(l2)

56